# Connect with Google Drive using Google API
Other library: https://pythonhosted.org/PyDrive/quickstart.html

In [14]:
from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [15]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

def main():
    """
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # token.json stores the user's access and refresh tokens, and is created
    # automatically when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('drive', 'v3', credentials=creds)

        # Call the Drive v3 API
        results = service.files().list(
            pageSize=10, fields="nextPageToken, files(id, name)").execute()
        items = results.get('files', [])

        if not items:
            print('No files found.')
            return
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')

if __name__ == '__main__':
    main()

Files:
Hi.txt (1EpJY4maoG6JImPjjd1T2r1iGRrOlyKLR)
Sztudia (0B2R7hpBSo46OZ2hVNHlSeUlFRkk)
IT_Analisis_Data (1cTE2fLpGwIhmTQsJ76hQexSAIEaPM5EZ)
jobData.csv (1Jk_6f3D0J27dVdinbfaBhuyApw3VE_VF)
Lista osób wesele 2024r. (1Y3VnCTkKwnWdGpkgy_ED0DjRYF-psIFxCOaNAuMdwz0)
WSI pogrupowane (1YhU6XGBKupPgbhbAicf8l1RUX9GwbCS-OsF8SOXJnWw)
projects_cpp.jpg (1nXTfl6kxfOTm1GdThNss5uizsZnjvFRC)
projects_python.jpg (1BdJ8BoAcpTfAn_nxfRZBXt7SGLwu1C85)
economics_in2.JPG (13kgHpTxjoQaGPvvmnuYDLBbO7eNG5syy)
WebPage_photos (1aUV97lHM0xj3vjicZGaBnB-uf3l6Mn71)


# Work with CSV

In [17]:
import csv
import io
import os.path
import pandas as pd  # Dodajemy obsługę pandas

SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
file_id = '1Jk_6f3D0J27dVdinbfaBhuyApw3VE_VF'

def download_csv():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('drive', 'v3', credentials=creds)

        # Download CSV
        file = service.files().get(fileId=file_id).execute()
        download_url = file.get('webContentLink')
        if download_url:
            resp, content = service.http.request(download_url)
            return content
        else:
            print('File not found.')
            return None

    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')
        return None

def process_csv(content):
    if content:
        # Tworzenie obiektu IO do odczytu zawartości pliku jako CSV
        csv_io = io.StringIO(content.decode('utf-8'))

        # Odczytywanie zawartości pliku CSV
        csv_reader = csv.reader(csv_io)

        # Tworzenie ramki danych pandasowej
        df = pd.DataFrame(csv_reader)

        # Wyświetlanie ramki danych
        print(df)

if __name__ == '__main__':
    csv_content = download_csv()
    if csv_content:
        process_csv(csv_content)

File not found.
